In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cust_demo = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_01_DEMO.csv')
cust_buy = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_02_PDDE.csv')
aff_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_03_COP_U.csv')
goods_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_04_PD_CLAC.csv')
store_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_05_BR.csv')
Lpay_info = pd.read_csv('C:/Users/legg0/Desktop/롯데멤버쉽 공모전/Data/LPOINT_BIG_COMP_06_LPAY.csv')

C:\Users\legg0\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Dataset info 확인

* cust : 고객의 고유 식별번호
* ma_fem_dv : 남성, 여성
* ages : 20대 ~ 70대 이상
* zon_hlv : 지역 대분류 코드
* zom_mcls: 지역 중분류 코드
* rct_no : 장바구니 식별 번호(구매 고유 식별번호)
* chnl_dv : 온라인/오프라인 구분 (1: 오프라인 / 2: 온라인)
* pd_c : 상품코드
* pd_nm: 상품 명
* clac_hlv_nm : 상품 카테고리 대분류명
* clac_mcls_nm : 상품 카테고리 중분류명
* br_c : 구매 점포 구분 코드
* de_dt : YYYY/MM/DD
* de_hr : HH
* buy_am : 구매한 금액
* buy_ct : 구매한 수량
* cop_c : A01-06(유통사), B01(숙박업종), C01-2(엔터테인먼트), D01~2: F&B, E01: 렌탈업종, L00(기타제휴사), L01(비제휴사)

In [3]:
cust_demo

,cust,ma_fem_dv,ages,zon_hlv
0,M000034966,여성,40대,Z07
1,M000059535,여성,30대,Z12
2,M000136117,여성,30대,Z11
3,M000201112,여성,50대,Z17
4,M000225114,여성,40대,Z17
...,...,...,...,...
29908,M999708287,여성,40대,Z10
29909,M999770689,여성,30대,Z16
29910,M999849895,여성,20대,Z04
29911,M999926092,남성,30대,Z08


In [98]:
cust_buy.isnull().sum().sort_values(ascending = False)

br_c       375784
buy_ct          0
buy_am          0
de_hr           0
de_dt           0
pd_c            0
cop_c           0
chnl_dv         0
rct_no          0
cust            0
dtype: int64

In [5]:
store_info

,br_c,cop_c,zon_hlv,zon_mcls
0,A010001,A01,Z17,Z17024
1,A010002,A01,Z17,Z17018
2,A010003,A01,Z17,Z17011
3,A010004,A01,Z16,Z16007
4,A010005,A01,Z17,Z17005
...,...,...,...,...
8803,E010040,E01,Z17,Z17024
8804,E010041,E01,Z16,Z16005
8805,E010042,E01,Z17,Z17021
8806,E010043,E01,Z10,Z10035


In [6]:
goods_info

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,PD0001,소파,가구,거실가구
1,PD0002,스툴/리빙의자,가구,거실가구
2,PD0003,탁자,가구,거실가구
3,PD0004,장식장/진열장,가구,거실가구
4,PD0005,기타가구,가구,기타가구
...,...,...,...,...
1928,PD1929,여성향수,화장품/뷰티케어,향수
1929,PD1930,남성향수,화장품/뷰티케어,향수
1930,PD1931,남녀공용향수,화장품/뷰티케어,향수
1931,PD1932,남성향수세트,화장품/뷰티케어,향수


In [7]:
Lpay_info

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
0,M629656521,210803210311226,A03,1,20210803,21,10900
1,M216016456,210803130167542,L01,2,20210803,13,6860
2,M205142844,210803140275112,A02,1,20210803,14,9000
3,M737010483,210803040637594,A06,2,20210803,4,36740
4,M707775545,210803140675502,A06,2,20210803,14,138500
...,...,...,...,...,...,...,...
353179,M470112503,210125110644545,A06,2,20210125,11,51800
353180,M470112503,210104210652271,A06,2,20210104,21,6440
353181,M748878049,210119200278240,A02,1,20210119,20,17600
353182,M748878049,210119180267206,A02,1,20210119,18,12400


* 유통사 제외 엘페이 Rows 제거
* 새로운 Df 정의 (엘페이)

In [8]:
cop = Lpay_info['cop_c'].tolist()
cust_cop = []

for i in range(len(cop)):
    if cop[i][0] != 'A':
        cust_cop.append(i)

Lpay_info.drop(cust_cop, inplace = True, axis = 0)
Lpay_info.reset_index(drop = True, inplace=True)
Lpay_info

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
0,M629656521,210803210311226,A03,1,20210803,21,10900
1,M205142844,210803140275112,A02,1,20210803,14,9000
2,M737010483,210803040637594,A06,2,20210803,4,36740
3,M707775545,210803140675502,A06,2,20210803,14,138500
4,M848083175,210803180197432,A01,1,20210803,18,14000
...,...,...,...,...,...,...,...
227992,M470112503,210104210651243,A06,2,20210104,21,50920
227993,M470112503,210125110644545,A06,2,20210125,11,51800
227994,M470112503,210104210652271,A06,2,20210104,21,6440
227995,M748878049,210119200278240,A02,1,20210119,20,17600


* 고객 정보와 엘페이 df의 결합

In [9]:
Lpay = pd.merge(Lpay_info, cust_demo, on = 'cust', how = 'left')
Lpay['Lpay_use'] = 1
Lpay

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,Lpay_use
0,M629656521,210803210311226,A03,1,20210803,21,10900,남성,40대,Z04,1
1,M205142844,210803140275112,A02,1,20210803,14,9000,여성,60대,Z17,1
2,M737010483,210803040637594,A06,2,20210803,4,36740,남성,40대,Z16,1
3,M707775545,210803140675502,A06,2,20210803,14,138500,남성,30대,Z11,1
4,M848083175,210803180197432,A01,1,20210803,18,14000,남성,20대,Z10,1
...,...,...,...,...,...,...,...,...,...,...,...
227992,M470112503,210104210651243,A06,2,20210104,21,50920,여성,50대,Z16,1
227993,M470112503,210125110644545,A06,2,20210125,11,51800,여성,50대,Z16,1
227994,M470112503,210104210652271,A06,2,20210104,21,6440,여성,50대,Z16,1
227995,M748878049,210119200278240,A02,1,20210119,20,17600,여성,30대,Z10,1


In [10]:
Lpay[Lpay['cop_c'] == 'A06']

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,Lpay_use
2,M737010483,210803040637594,A06,2,20210803,4,36740,남성,40대,Z16,1
3,M707775545,210803140675502,A06,2,20210803,14,138500,남성,30대,Z11,1
6,M617097623,210803110658511,A06,2,20210803,11,42450,여성,40대,Z10,1
18,M754175163,210803100653431,A06,2,20210803,10,17540,여성,30대,Z04,1
22,M933302304,210803050638234,A06,2,20210803,5,70050,여성,30대,Z17,1
...,...,...,...,...,...,...,...,...,...,...,...
227987,M688241278,210125110643662,A06,2,20210125,11,27120,남성,40대,Z17,1
227991,M920715369,210110150608046,A06,2,20210110,15,24000,여성,40대,Z17,1
227992,M470112503,210104210651243,A06,2,20210104,21,50920,여성,50대,Z16,1
227993,M470112503,210125110644545,A06,2,20210125,11,51800,여성,50대,Z16,1


* 유통사 Lpay 사용: 227997
* 유통사 제외 Rows 모두 제거
* 다시 병합하고, 전처리 하기

In [11]:
Lpay.rename(columns = {'rct_no' : 'Lpay_rct_no'}, inplace = True)
Lpay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227997 entries, 0 to 227996
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   cust         227997 non-null  object
 1   Lpay_rct_no  227997 non-null  int64 
 2   cop_c        227997 non-null  object
 3   chnl_dv      227997 non-null  int64 
 4   de_dt        227997 non-null  int64 
 5   de_hr        227997 non-null  int64 
 6   buy_am       227997 non-null  int64 
 7   ma_fem_dv    227997 non-null  object
 8   ages         227997 non-null  object
 9   zon_hlv      227997 non-null  object
 10  Lpay_use     227997 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 20.9+ MB


In [12]:
# real_df.groupby(['Lpay_rct_no'])['buy_ct'].sum()

## Preprocessing

* 유통사 Data 중복 제거
* 유통사 + 고객

In [13]:
cust_buy = cust_buy.drop_duplicates()
cust_buy_demo = pd.merge(cust_buy, cust_demo, on = 'cust', how = 'left')
cust_buy_demo

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,여성,50대,Z17
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,여성,40대,Z13
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1,여성,50대,Z17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144384,M816318679,E06052115831,2,A06,NaN,PD0507,20211231,22,50000.0,1,여성,40대,Z17
4144385,M816318679,E06052116037,2,A06,NaN,PD0507,20211231,22,10000.0,1,여성,40대,Z17
4144386,M182645944,E06052117103,2,A06,NaN,PD0294,20211231,23,289000.0,1,남성,20대,Z17
4144387,M533286446,E06052118403,2,A06,NaN,PD0507,20211231,23,200000.0,1,남성,40대,Z10


* 4381743 -> 4144389 : 약 20만개 중복 데이터 제거

* 유통사 + 고객 + 상품

In [14]:
cust_goods = pd.merge(cust_buy_demo, goods_info, on = 'pd_c', how = 'left')
cust_goods

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,여성,40대,Z13,기타주방일회용품,주방잡화,주방일회용품
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1,여성,50대,Z17,커피/음료,테넌트/음식점,식당
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144384,M816318679,E06052115831,2,A06,NaN,PD0507,20211231,22,50000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권
4144385,M816318679,E06052116037,2,A06,NaN,PD0507,20211231,22,10000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권
4144386,M182645944,E06052117103,2,A06,NaN,PD0294,20211231,23,289000.0,1,남성,20대,Z17,남성가디건,남성의류,남성의류상의
4144387,M533286446,E06052118403,2,A06,NaN,PD0507,20211231,23,200000.0,1,남성,40대,Z10,백화점/마트모바일상품권,상품권,모바일상품권


* 유통사 + 고객 + 상품 + 점포

In [15]:
df_cust = pd.merge(cust_goods, store_info, on = ['br_c', 'cop_c', 'zon_hlv'], how = 'left')
df_cust

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,여성,40대,Z13,기타주방일회용품,주방잡화,주방일회용품,Z13001
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1,여성,50대,Z17,커피/음료,테넌트/음식점,식당,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144384,M816318679,E06052115831,2,A06,NaN,PD0507,20211231,22,50000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권,NaN
4144385,M816318679,E06052116037,2,A06,NaN,PD0507,20211231,22,10000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권,NaN
4144386,M182645944,E06052117103,2,A06,NaN,PD0294,20211231,23,289000.0,1,남성,20대,Z17,남성가디건,남성의류,남성의류상의,NaN
4144387,M533286446,E06052118403,2,A06,NaN,PD0507,20211231,23,200000.0,1,남성,40대,Z10,백화점/마트모바일상품권,상품권,모바일상품권,NaN


In [96]:
df_cust.isnull().sum().sort_values(ascending = False)

zon_mcls        957095
de_hr                0
rct_no               0
chnl_dv              0
cop_c                0
br_c                 0
pd_c                 0
de_dt                0
buy_am               0
clac_mcls_nm         0
buy_ct               0
ma_fem_dv            0
ages                 0
zon_hlv              0
pd_nm                0
clac_hlv_nm          0
cust                 0
dtype: int64

In [16]:
df_cust['br_c'].fillna(0, inplace = True)

In [17]:
df_cust['zon_hlv'].value_counts()

Z17    1184096
Z10    1099778
Z16     329074
Z04     266889
Z11     266787
Z06     172762
Z13     133001
Z05     106938
Z08      96046
Z09      91097
Z15      85061
Z14      82728
Z12      71731
Z03      66248
Z01      61652
Z07      22305
Z02       8196
Name: zon_hlv, dtype: int64

In [18]:
df_cust[df_cust['cop_c'] == 'A06']

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls
4078035,M466487936,E06000092239,2,A06,0,PD0390,20210101,16,1000.0,1,여성,40대,Z10,쌈/또띠아,대용식,건면,NaN
4078036,M552902613,E06000099529,2,A06,0,PD1776,20210101,0,157000.0,1,여성,40대,Z11,여성펌프스,패션잡화,여성화,NaN
4078037,M102283139,E06000100696,2,A06,0,PD1022,20210101,0,32900.0,1,여성,40대,Z10,피규어,완구,남아완구,NaN
4078038,M803112421,E06000101467,2,A06,0,PD1889,20210101,0,54000.0,1,여성,30대,Z17,BB/파운데이션/컴팩트류,화장품/뷰티케어,메이크업,NaN
4078039,M803112421,E06000101467,2,A06,0,PD1893,20210101,0,34000.0,1,여성,30대,Z17,마스카라,화장품/뷰티케어,메이크업,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144384,M816318679,E06052115831,2,A06,0,PD0507,20211231,22,50000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권,NaN
4144385,M816318679,E06052116037,2,A06,0,PD0507,20211231,22,10000.0,1,여성,40대,Z17,백화점/마트모바일상품권,상품권,모바일상품권,NaN
4144386,M182645944,E06052117103,2,A06,0,PD0294,20211231,23,289000.0,1,남성,20대,Z17,남성가디건,남성의류,남성의류상의,NaN
4144387,M533286446,E06052118403,2,A06,0,PD0507,20211231,23,200000.0,1,남성,40대,Z10,백화점/마트모바일상품권,상품권,모바일상품권,NaN


* 유통사와 엘페이의 병합
* 엘페이와 중복되는 값 제거필요
* merge -> 엘페이와 유통사의 inner join 없애고, outer로 merge
* 하나의 df 완성

###  병합

#### Inner Merge

In [19]:
df1 = pd.merge(Lpay, df_cust, on = ['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am', 'ma_fem_dv', 'ages', 'zon_hlv'], how = 'inner')
df1_idx = df1[df1['Lpay_use'] == 1].index
Lpay.drop(df1_idx, inplace = True)
Lpay.reset_index(drop = True, inplace=True)

In [20]:
df1

,cust,Lpay_rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,Lpay_use,rct_no,br_c,pd_c,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls
0,M205142844,210803140275112,A02,1,20210803,14,9000,여성,60대,Z17,1,A02045353147,A020008,PD0819,3,여성스포츠티셔츠/탑,스포츠패션,여성일반스포츠의류,Z17011
1,M425728822,210803170188958,A01,1,20210803,17,12000,여성,30대,Z10,1,A01292600611,A010060,PD0288,1,임대매출,기타(비상품),임대매출,NaN
2,M425728822,210803180198612,A01,1,20210803,18,24000,여성,30대,Z10,1,A01292769549,A010060,PD0288,1,임대매출,기타(비상품),임대매출,NaN
3,M314794317,210803120161056,A01,1,20210803,12,3000,여성,30대,Z17,1,A01291899962,A010001,PD0232,1,일반스낵,과자,스낵류,Z17024
4,M895829724,210803130165989,A01,1,20210803,13,10500,여성,30대,Z17,1,A01292031621,A010017,PD1694,1,푸드코트한식,테넌트/음식점,푸드코트,Z17009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37566,M688241278,210120180268758,A02,1,20210120,18,3600,남성,40대,Z17,1,A02041327001,A020008,PD1690,1,베이커리,테넌트/음식점,식당,Z17011
37567,M688241278,210130180166109,A01,1,20210130,18,10000,남성,40대,Z17,1,A01031417249,A010003,PD1694,1,푸드코트한식,테넌트/음식점,푸드코트,Z17011
37568,M688241278,210119140144047,A01,1,20210119,14,11900,남성,40대,Z17,1,A01019375400,A010003,PD1686,1,중식,테넌트/음식점,식당,Z17011
37569,M688241278,210110150112436,A01,1,20210110,15,11900,남성,40대,Z17,1,A01010333816,A010003,PD1686,1,중식,테넌트/음식점,식당,Z17011


In [21]:
Lpay

,cust,Lpay_rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,Lpay_use
0,M224473145,210517030614283,A06,2,20210517,3,132564,여성,40대,Z06,1
1,M224473145,210515150663884,A06,2,20210515,15,27970,여성,40대,Z06,1
2,M224473145,210515150664312,A06,2,20210515,15,30960,여성,40대,Z06,1
3,M224473145,210515150665525,A06,2,20210515,15,83010,여성,40대,Z06,1
4,M224473145,210512220622734,A06,2,20210512,22,60440,여성,40대,Z06,1
...,...,...,...,...,...,...,...,...,...,...,...
190421,M470112503,210104210651243,A06,2,20210104,21,50920,여성,50대,Z16,1
190422,M470112503,210125110644545,A06,2,20210125,11,51800,여성,50대,Z16,1
190423,M470112503,210104210652271,A06,2,20210104,21,6440,여성,50대,Z16,1
190424,M748878049,210119200278240,A02,1,20210119,20,17600,여성,30대,Z10,1


#### Outer Merge

In [75]:
real_df = pd.merge(df_cust, Lpay, on = ['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am',
       'ma_fem_dv', 'ages', 'zon_hlv'], how = 'outer')
real_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls,Lpay_rct_no,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,기타주방일회용품,주방잡화,주방일회용품,Z13001,NaN,NaN
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성티셔츠,남성의류,남성의류상의,NaN,NaN,NaN
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,커피/음료,테넌트/음식점,식당,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304976,M470112503,NaN,2,A06,NaN,NaN,20210104,21,50920.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101042e+14,1.0
4304977,M470112503,NaN,2,A06,NaN,NaN,20210125,11,51800.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101251e+14,1.0
4304978,M470112503,NaN,2,A06,NaN,NaN,20210104,21,6440.0,NaN,여성,50대,Z16,NaN,NaN,NaN,NaN,2.101042e+14,1.0
4304979,M748878049,NaN,1,A02,NaN,NaN,20210119,20,17600.0,NaN,여성,30대,Z10,NaN,NaN,NaN,NaN,2.101192e+14,1.0


In [76]:
real_df.isnull().sum().sort_values(ascending = False)

Lpay_use        4113562
Lpay_rct_no     4113562
zon_mcls        1117233
rct_no           159928
br_c             159928
pd_c             159928
buy_ct           159928
pd_nm            159928
clac_hlv_nm      159928
clac_mcls_nm     159928
ages                  0
ma_fem_dv             0
zon_hlv               0
buy_am                0
de_hr                 0
de_dt                 0
cop_c                 0
chnl_dv               0
cust                  0
dtype: int64

### 전처리

#### Target Setting

In [77]:
real_df.columns

Index(['cust', 'rct_no', 'chnl_dv', 'cop_c', 'br_c', 'pd_c', 'de_dt', 'de_hr',
       'buy_am', 'buy_ct', 'ma_fem_dv', 'ages', 'zon_hlv', 'pd_nm',
       'clac_hlv_nm', 'clac_mcls_nm', 'zon_mcls', 'Lpay_rct_no', 'Lpay_use'],
      dtype='object')

In [78]:
real_df.drop(['Lpay_rct_no', 'pd_nm', 'clac_mcls_nm', 'zon_mcls'], axis = 1, inplace = True)

In [79]:
real_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,NaN
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,NaN
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,NaN
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304976,M470112503,NaN,2,A06,NaN,NaN,20210104,21,50920.0,NaN,여성,50대,Z16,NaN,1.0
4304977,M470112503,NaN,2,A06,NaN,NaN,20210125,11,51800.0,NaN,여성,50대,Z16,NaN,1.0
4304978,M470112503,NaN,2,A06,NaN,NaN,20210104,21,6440.0,NaN,여성,50대,Z16,NaN,1.0
4304979,M748878049,NaN,1,A02,NaN,NaN,20210119,20,17600.0,NaN,여성,30대,Z10,NaN,1.0


* 결측치 있는 값 행 제거

In [80]:
real_df['Lpay_use'].fillna(0, inplace = True)
real_df['Lpay_use'] = real_df['Lpay_use'].astype(int)

In [81]:
real_df = real_df.dropna(axis = 0)
real_df.reset_index()

,index,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,0
1,1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,0
2,2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
3,3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
4,4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145048,4145048,M816318679,E06052115831,2,A06,0,PD0507,20211231,22,50000.0,1.0,여성,40대,Z17,상품권,0
4145049,4145049,M816318679,E06052116037,2,A06,0,PD0507,20211231,22,10000.0,1.0,여성,40대,Z17,상품권,0
4145050,4145050,M182645944,E06052117103,2,A06,0,PD0294,20211231,23,289000.0,1.0,남성,20대,Z17,남성의류,0
4145051,4145051,M533286446,E06052118403,2,A06,0,PD0507,20211231,23,200000.0,1.0,남성,40대,Z10,상품권,0


In [82]:
real_df['Lpay_use']

0          0
1          0
2          0
3          0
4          0
          ..
4145048    0
4145049    0
4145050    0
4145051    0
4145052    0
Name: Lpay_use, Length: 4145053, dtype: int32

In [83]:
real_df['cop_c'].value_counts().sort_values(ascending = False)

A02    2241885
A03     858568
A01     670525
A04     299779
A06      66426
A05       7870
Name: cop_c, dtype: int64

In [84]:
real_df.isnull().sum().sort_values(ascending = False)

Lpay_use       0
clac_hlv_nm    0
zon_hlv        0
ages           0
ma_fem_dv      0
buy_ct         0
buy_am         0
de_hr          0
de_dt          0
pd_c           0
br_c           0
cop_c          0
chnl_dv        0
rct_no         0
cust           0
dtype: int64

---------------------------------------------------------------------------------------결측치 제거 완료 ---------------------------------------------------------------------------------

* 영수증 중복 처리 -> O
* 전체 컬럼 전처리 -> 
* 샘플링
* Modeling

-- df -> csv 저장 --

In [85]:
tmp_df = real_df

tmp_df = tmp_df[tmp_df.duplicated(subset = 'rct_no')].reset_index()
####### tmp_df[조건, bool값]

In [86]:
tmp_df

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,여성,50대,Z17,남성의류,0
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,여성,40대,Z13,주방잡화,0
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,여성,50대,Z17,남성의류,0
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1.0,여성,50대,Z17,테넌트/음식점,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145048,M816318679,E06052115831,2,A06,0,PD0507,20211231,22,50000.0,1.0,여성,40대,Z17,상품권,0
4145049,M816318679,E06052116037,2,A06,0,PD0507,20211231,22,10000.0,1.0,여성,40대,Z17,상품권,0
4145050,M182645944,E06052117103,2,A06,0,PD0294,20211231,23,289000.0,1.0,남성,20대,Z17,남성의류,0
4145051,M533286446,E06052118403,2,A06,0,PD0507,20211231,23,200000.0,1.0,남성,40대,Z10,상품권,0


In [87]:
tmp_df[tmp_df['Lpay_use'] == 1]

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,clac_hlv_nm,Lpay_use
22,M686795542,A01000022861,1,A01,A010045,PD0952,20210101,11,34800.0,1.0,여성,20대,Z16,여성의류,1
73,M049345995,A01000057303,1,A01,A010043,PD1827,20210101,13,39900.0,2.0,여성,20대,Z10,패션잡화,1
108,M502480776,A01000087195,1,A01,A010045,PD1692,20210101,14,5500.0,1.0,여성,30대,Z03,테넌트/음식점,1
120,M049345995,A01000095023,1,A01,A010043,PD1699,20210101,14,43000.0,1.0,여성,20대,Z10,테넌트/음식점,1
135,M030477840,A01000105587,1,A01,A010045,PD0298,20210101,15,109400.0,1.0,여성,30대,Z16,남성의류,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144968,M324896636,E06052060518,2,A06,0,PD0507,20211229,16,50000.0,1.0,남성,20대,Z17,상품권,1
4144969,M305766997,E06052061811,2,A06,0,PD0511,20211229,16,9800.0,1.0,남성,40대,Z17,상품권,1
4144970,M305766997,E06052061811,2,A06,0,PD0511,20211229,16,9800.0,1.0,남성,40대,Z17,상품권,1
4144971,M305766997,E06052062286,2,A06,0,PD0511,20211229,16,9800.0,1.0,남성,40대,Z17,상품권,1


In [88]:
tmp_df['chnl_dv'].value_counts()

1    3769189
2     375864
Name: chnl_dv, dtype: int64

------------------------------------------------------DF -> CSV-------------------------------------------------------

### 결측치확인

In [35]:
np.mean(real_df[real_df['buy_ct'] == 1]['buy_am'])

26167.98351071982

In [36]:
df1['pd_nm'].value_counts()[:20]

임대매출       2253
디저트        1531
한식         1440
푸드코트한식     1384
커피/음료      1177
일반스낵       1027
수입담배        982
국산담배        887
베이커리일반빵     881
쿠키          732
커피음료        699
일식          671
양식          610
기타레토르트      540
일반빵         485
베이커리        485
동남아식        480
푸드코트컨세션     479
분식          464
여성티셔츠/탑     311
Name: pd_nm, dtype: int64

In [100]:
real_df['clac_hlv_nm'].value_counts()[:20]

채소            414891
과자            404103
대용식           281011
유제품           272141
냉장식품          248730
음료            231893
과일            224529
축산물           186103
테넌트/음식점       162054
주류            159566
조미료           134819
냉동식품          113192
청소/세탁/욕실용품    103131
조리식품           98978
수산물            77066
문구/사무용품        75541
여성의류           71253
주방잡화           58998
세제/위생          58146
스포츠패션          58058
Name: clac_hlv_nm, dtype: int64

In [102]:
real_df['clac_hlv_nm'].value_counts()[:20]

채소            414891
과자            404103
대용식           281011
유제품           272141
냉장식품          248730
음료            231893
과일            224529
축산물           186103
테넌트/음식점       162054
주류            159566
조미료           134819
냉동식품          113192
청소/세탁/욕실용품    103131
조리식품           98978
수산물            77066
문구/사무용품        75541
여성의류           71253
주방잡화           58998
세제/위생          58146
스포츠패션          58058
Name: clac_hlv_nm, dtype: int64

In [39]:
df1['br_c'].value_counts()[:20]

A010002    3742
A010001    2502
0          1443
A010004    1436
A010026    1170
A010017    1105
A010034    1009
A010037     842
A010060     776
A010062     742
A010045     704
A010041     614
A010015     564
A010033     558
A010032     488
A020013     486
A010063     464
A010020     458
A010008     452
A010042     420
Name: br_c, dtype: int64

In [40]:
real_df['pd_c'].value_counts()[:20]

PD1484    88162
PD0232    87994
PD1156    79174
PD0403    57913
PD0480    55967
PD1387    53371
PD1486    51988
PD0228    48812
PD1330    45448
PD0413    42940
PD0197    39541
PD1173    37866
PD0218    37820
PD1155    37309
PD1333    36665
PD0230    34029
PD1152    33526
PD0288    33499
PD1153    32804
PD1331    31575
Name: pd_c, dtype: int64

* Lpay결제 여부이니까 영수증 groupby해주면 좋음
* Lpay를 사용하는 데 있어서 제품 구매가 큰 영향을 줄까?
* 개인화(고객)의 영역에 있어서는 개인화 마케팅에 초점 맞춰서 그럴 수 있다 판단함
* 다른 컬럼에 대해서 어떤 컬럼이 큰 영향을 줄까?

In [52]:
tmp = tmp_df.duplicated(subset = ['rct_no'])
tmp

0          False
1          False
2          False
3          False
4          False
           ...  
4145048    False
4145049    False
4145050    False
4145051    False
4145052    False
Length: 4145053, dtype: bool

 * 어짜피 Lpay 사용 여부 판단 모델이니 그룹바이해서 한사람이 한 번 결제하는 거니 묶어버림
 * 120만개 target -> 0
 * 6만개 target -> 1
 * 오버 샘플링 

In [42]:
real_df.columns

Index(['cust', 'rct_no', 'chnl_dv', 'cop_c', 'br_c', 'pd_c', 'de_dt', 'de_hr',
       'buy_am', 'buy_ct', 'ma_fem_dv', 'ages', 'zon_hlv', 'clac_hlv_nm',
       'Lpay_use'],
      dtype='object')

* groupby 해주면 120만개로 확줄어드는 것 확인할 수 있음
* target값 Lpay는 중복없음 확인함 20만개 
* 100만: 20만
* 결측치 처리만 어떻게하면 좋은 Model 만들 수 있을 것 같은데
* 제품 3개 중 하나 또는 두개만 남겨도 되지 않을까?
* 데이터를 어느 정도 가져갈 것이며, 결측치는 어떻게 해결해야 할지 감이안옴


* 하나 대안은 언더샘플링 진행하고, 홀드 아웃 통해서 모델 성능 올리는 방안

## Modeling

* Sampling 어떻게 할지 -> 
*

In [65]:
pip install imbalanced-learn

  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\legg0\anaconda3\python.exe -m pip install --upgrade pip' command.


In [68]:
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

In [91]:
X = tmp_df.drop('Lpay_use', axis = 1)
Y = tmp_df['Lpay_use']

In [92]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

In [93]:
train_Y.value_counts()

0    3085288
1      23501
Name: Lpay_use, dtype: int64

In [95]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *

KNN_model = KNN(n_neighbors = 11).fit(train_X, train_Y)
pred_Y = KNN_model.predict(test_X)
print(recall_score(test_Y, pred_Y))
print(accuracy_score(test_Y, pred_Y))

ValueError: could not convert string to float: 'M857190643'

In [ ]:
undersample = RandomUnderSampler(sampling_strategy='majority')

In [ ]:
one-hot -> 태준이 모델의 문제점은 클래스 불균형 맞추고 각자 모델 돌립시다
지금 하기로한 데이터
건하씨 csv 